<a href="https://colab.research.google.com/github/sydoky/Recession-Analysis/blob/main/Recession_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio

data=pd.read_csv("/content/UK_monthly_gdp.csv")
data.head(10)

,Time Period,GDP Growth
0,/01/2020,0.3
1,/02/2020,-0.5
2,/03/2020,-7.0
3,/04/2020,-20.9
4,/05/2020,3.2
5,/06/2020,9.0
6,/07/2020,7.4
7,/08/2020,2.0
8,/09/2020,1.1
9,/10/2020,0.6


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Time Period  36 non-null     object 
 1   GDP Growth   36 non-null     float64
dtypes: float64(1), object(1)
memory usage: 704.0+ bytes


# **Let's have a look at th GDP growth over time:**

In [3]:
fig=go.Figure(data=go.Heatmap(z=[data["GDP Growth"]],
              x=data.index,
              y=["GDP Growth"],
              colorscale="Viridis"))
fig.update_layout(title="GDP Growth over Time",
                  xaxis_title="Time Period",
                  yaxis_title="")

fig.show()
#time period represents the months

Recession means the decline in the circulation of money for two consecutive quarters

# **That's why we have to convert our monthly data into quarterly data to analyze the recession:**

In [4]:
data["Time Period"]=pd.to_datetime(data["Time Period"],format="/%m/%Y")
data.set_index("Time Period",inplace=True)
quarterly_data=data.resample("Q").mean()

quarterly_data.head(10)

,GDP Growth
Time Period,
2020-03-31,-2.400000
2020-06-30,-2.900000
2020-09-30,3.500000
2020-12-31,0.200000
2021-03-31,0.033333
2021-06-30,1.800000
2021-09-30,0.433333
2021-12-31,0.400000
2022-03-31,0.066667


# **Let's Calculate and analyze recession based on quarterly GDP Growth:**

In [5]:
quarterly_data["Recession"]=((quarterly_data["GDP Growth"]<0)&(quarterly_data["GDP Growth"].shift(1)<0))

#Fill Missing values with False

quarterly_data["Recession"].fillna(False,inplace=True)

#Plot The GDP growth and recession data

fig=go.Figure()
fig.add_trace(go.Scatter(x=quarterly_data.index,
                         y=quarterly_data["GDP Growth"],
                         name="GDP Growth",
                         line=dict(color="green",width=2)))
fig.add_trace(go.Scatter(x=quarterly_data[quarterly_data["Recession"]].index,
                         y=quarterly_data[quarterly_data["Recession"]]["GDP Growth"],
                         name="Recession",line=dict(color="red",width=2)))

fig.update_layout(title="GDP Growth and Recession Over Time(Quarterly Data",
                  xaxis_title="Time Period",
                  yaxis_title="GDP Growth")

fig.show()


In [6]:
quarterly_data["Recession"]

Time Period
2020-03-31    False
2020-06-30     True
2020-09-30    False
2020-12-31    False
2021-03-31    False
2021-06-30    False
2021-09-30    False
2021-12-31    False
2022-03-31    False
2022-06-30    False
2022-09-30     True
2022-12-31    False
Freq: Q-DEC, Name: Recession, dtype: bool

# **Let's analyze The severity of the Recession**

In [7]:
quarterly_data["Recession Start"]=quarterly_data["Recession"].ne(quarterly_data["Recession"].shift()).cumsum()

recession_periods= quarterly_data.groupby("Recession Start")

recession_duration = recession_periods.size()

recession_severity = recession_periods["GDP Growth"].sum()

fig=go.Figure()

fig.add_trace(go.Bar(x=recession_duration.index,y=recession_duration,name="Recession Duration"))

fig.add_trace(go.Bar(x=recession_severity.index,y=recession_severity,name="Recession Severity"))

fig.update_layout(title="Duration and Severity of Recession",
                  xaxis_title="Recession Periods",
                  yaxis_title="Duration/Severity")

fig.show()